## FIBSEM Database (SQLModel Version)

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.db.v3 import models

user = models.User(username='patrick', 
                name='Patrick Cleeve',
                email = 'patrick@openfibsem.org',
                password = 'password',
                role = 'admin')

print(user.username)




In [ ]:
from sqlmodel import Session, SQLModel, create_engine, select
from fibsem.db.v3 import models
from fibsem import config as cfg

def create_connection(database_path: str = cfg.DATABASE_PATH, echo: bool = False):
    # create a connection to the database
    engine = create_engine(f'sqlite:///{database_path}', echo=echo)

    return engine

def create_database(engine):
    # create the database
    SQLModel.metadata.create_all(engine)

engine = create_connection(echo=True)
create_database(engine)

# with Session(engine) as session:
    # session.add(user)
    # session.commit()

In [ ]:

def get_users(session):
    with session:
        statement = select(models.User)
        users = session.exec(statement)
        return [user for user in users]


def get_projects(session):
    with session:
        statement = select(models.Project)
        projects = session.exec(statement)
        return [project for project in projects]

def get_samples(session):
    with session:
        statement = select(models.Sample)
        samples = session.exec(statement)
        return [sample for sample in samples]
    
def get_instruments(session):
    with session:
        statement = select(models.Instrument)
        instruments = session.exec(statement)
        return [instrument for instrument in instruments]


def get_user(session, user_id):
    with session:
        statement = select(models.User).where(models.User.id == user_id)
        user = session.exec(statement).first()
        return user

engine = create_connection(echo=True)
session = Session(engine)

In [ ]:
# get all users

users = get_users(session)
for user in users:
    print(user.username)


In [ ]:


# upload experiment to database

# get user
users = get_users(session)
if len(users) > 0:
    print()
    print(f"Please select a user. The following users are available:")
    for user in users:
        print(f"{user.id}: {user.username} ({user.name}) - {user.email}")

# give option to create a user or select a user
create_user = input("Would you like to create a new user? (y/n): ")

if create_user.lower() == 'y':
    username = input("Enter the username: ")
    name = input("Enter the name: ")
    email = input("Enter the email: ")
    password = input("Enter the password: ")
    role = input("Enter the role: ")

    user = models.User(username=username, 
                    name=name,
                    email = email,
                    password = password,
                    role = role)

    with session:
        session.add(user)
        session.commit()

user_id = input("Enter the user id: ")
print(f"User id: {user_id}")

user = get_user(session, user_id)
print(user)

if user is None:
    print("User not found")
    exit()

# get project
projects = get_projects(session)

# give option to create a user or select a user
create_project = input("Would you like to create a new project? (y/n): ")

if len(projects) == 0 or create_project.lower() == 'y':
    print("No projects found... Creating a new project")

    name = input("Enter the project name: ")
    description = input("Enter the project description: ")

    project = models.Project(name=name, description=description, user_id=user_id)

    with session:
        session.add(project)
        session.commit()
    


else:

    print()
    print(f"Please select a project. The following projects are available:")
    for project in projects:
        print(f"{project.id}: {project.name} - {project.description}")

    project_id = input("Enter the project id: ")



# get sample


# get instrument


# extract dataframes from experiment
    